# Implementing Support Enumeration in Python: Hints

Import `NumPy` first of all.
(See [the lecture](http://lectures.quantecon.org/py/numpy.html) on `NumPy`.)

In [1]:
import numpy as np

Consider the example from von Stengel (2007):
$$
A =
\begin{bmatrix}
3 & 3 \\
2 & 5 \\
0 & 6
\end{bmatrix},
\quad
B =
\begin{bmatrix}
3 & 2 \\
2 & 6 \\
3 & 1
\end{bmatrix}.
$$

It will be convenient to denote the actions by *indices*,
so let the action spaces of players 1 and 2 be
$$
M = \{0, 1, 2\}, \quad
N = \{0, 1\}.
$$
(Note that in Python, indices start with 0.)

Create NumPy arrays for the payoff matrices:

In [2]:
A = np.array([[3, 3],
              [2, 5],
              [0 ,6]])
B = np.array([[3, 2],
              [2, 6],
              [3, 1]])

It will be more convenient to work with the [transpose](http://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.T.html) of matrix `B`:

In [3]:
B_T = B.T

In [4]:
B_T

array([[3, 2, 3],
       [2, 6, 1]])

## Support enumeration

For each $k = 1, \ldots, \min\{m, n\}$
and each support pair $(I, J)$ with $\lvert I\rvert = \lvert J\rvert = k$:

Solve the systems of linear equations:
$$
C \begin{pmatrix} y_J \\ u \end{pmatrix} = e
$$
and
$$
D \begin{pmatrix} x_I \\ v \end{pmatrix} = e
$$
with
$$
C =
 \begin{pmatrix}
 A_{IJ} & -\mathbf{1} \\
 \mathbf{1}' & 0
 \end{pmatrix}, \quad
D =
 \begin{pmatrix}
 B'_{JI} & -\mathbf{1} \\
 \mathbf{1}' & 0
 \end{pmatrix}, \quad
e = \begin{pmatrix}\mathbf{0} \\ 1\end{pmatrix},
$$
where
$A_{IJ}$ is the submatrix of $A$ given by rows $I$ and columns $J$,
$B'_{JI}$ is the submatrix of $B'$ given by rows $J$ and columns $I$, and
$\mathbf{0}$ and $\mathbf{1}$ are the $k$-dimensional vectors of zeros and ones,
respectively.

Then check that $x_i > 0$ for all $i \in I$ and $y_j > 0$ for all $j \in J$, and
that
$u \geq (A_J y_J)_i$ for all $i \notin I$ and
$v \geq (B'_I x_I)_j$ for all $j \notin J$,
where
$A_J$ is the submatrix of $A$ given by columns $J$ and
$B'_I$ is the submatrix of $B'$ given by columns $I$.

### Extracting a submatrix

Here's how to get the submatrix of $A$ with rows $1$ and $2$ and column $0$, for example.

Get the submarix $A_{[1, 2]}$ of $A$ with rows $1$ and $2$:

In [5]:
A_12 = A[[1, 2], :]

In [6]:
A_12

array([[2, 5],
       [0, 6]])

Get the submatrix $A_{[1, 2], [0]}$ of $A_{[1, 2]}$ with column $0$:

In [7]:
A_12_0 = A_12[:, [0]]

In [8]:
A_12_0

array([[2],
       [0]])

### Example: $I = \{0, 1\}$, $J = \{0, 1\}$

In [9]:
k = 2
I = [0, 1]
J = [0, 1]

Submatrix $A_{IJ}$:

In [10]:
A_IJ = A[I, :][:, J]

In [11]:
A_IJ

array([[3, 3],
       [2, 5]])

Submatrix $B'_{JI}$:

In [12]:
B_T_JI = B_T[J, :][:, I]

In [13]:
B_T_JI

array([[3, 2],
       [2, 6]])

Construct the matrices $C$ and $D$ and the vector $e$,
and pass them the linear equation solver [`np.linalg.solve`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html).